In [1]:
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
import torch.nn.functional as F
import tqdm.notebook as tqdm
from torch.utils.data import DataLoader
from transformers import BertTokenizerFast, BertForSequenceClassification
from transformers import AdamW

from utils.datasets import TextDataset
from models.bert import BertForClassification

In [2]:
MODEL = "neuralmind/bert-base-portuguese-cased"

# Loading tweet dataset

In [3]:
train_df = pd.read_csv("/home/kenzo/datasets/cleaned_tweetsentbr/train.tsv", sep="\t", names=["id", "label", "alfa", "text"], index_col=0)
test_df = pd.read_csv("/home/kenzo/datasets/cleaned_tweetsentbr/test.tsv", sep="\t", names=["id", "label", "alfa", "text"], index_col=0)

In [4]:
# TODO: Exploração dos dados

In [5]:
tokenizer = BertTokenizerFast.from_pretrained(MODEL)

In [6]:
# train_ds = BertDataset.from_df(train_df, tokenizer, max_seq_len=128)
# test_ds = BertDataset.from_df(test_df, tokenizer, max_seq_len=128)

train_ds = TextDataset.from_df(train_df, tokenizer, max_seq_len=128)
test_ds = TextDataset.from_df(test_df, tokenizer, max_seq_len=128)

# Preparing model

In [7]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
from functools import partial

metrics = {
    "accuracy": accuracy_score,
    "precision": partial(precision_score, average="macro"),
    "recall": partial(recall_score, average="macro"),
    "f1": partial(f1_score, average="macro"),
}

In [8]:
gpu = torch.device("cuda:1")

In [9]:
bert_model = BertForSequenceClassification.from_pretrained(MODEL, num_labels=3).cuda(gpu)

Some weights of the model checkpoint at neuralmind/bert-base-portuguese-cased were not used when initializing BertForSequenceClassification: ['cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.decoder.weight', 'cls.seq_relationship.weight', 'cls.seq_relationship.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPretraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the

In [10]:
model = BertForClassification(bert_model, metrics).cuda(gpu)

In [11]:
train_dl = DataLoader(train_ds, batch_size=64, shuffle=True)
test_dl = DataLoader(test_ds, batch_size=64, shuffle=True)

In [12]:
optimizer = AdamW(model.parameters(), lr=1e-5) 
criterion = nn.CrossEntropyLoss()

In [13]:
train, test = model.fit(3, train_dl, test_dl, criterion, optimizer, cuda=True, device=gpu)

- Remaining batches:   0%|          | 0/154 [00:00<?, ?it/s]	train_loss: 1.032899927783322 // test_loss: 0.8000206030332125// metrics: {'accuracy': 0.6311941510966693, 'precision': 0.6879866370804103, 'recall': 0.5769553547790643, 'f1': 0.5122388615526184}
Epoch: 2
- Remaining batches:   0%|          | 0/154 [00:00<?, ?it/s]	train_loss: 0.709916167444997 // test_loss: 0.6831468565341754// metrics: {'accuracy': 0.7047116165718927, 'precision': 0.6837611021892678, 'recall': 0.6876701179131954, 'f1': 0.6847516490355714}
Epoch: 3
- Remaining batches: 100%|██████████| 154/154 [07:27<00:00,  2.91s/it]
	train_loss: 0.6053705652812859 // test_loss: 0.6831144690513611// metrics: {'accuracy': 0.7087733549959383, 'precision': 0.6857082524137278, 'recall': 0.6848760550328631, 'f1': 0.683967037378174}


In [14]:
# torch.save(model.bert.state_dict(), "data/checkpoints/bert_tweetsent_br.ckpt")